## malaysian open data

In [ ]:
!pip install gtfs-realtime-bindings pandas requests
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import pandas as pd
from requests import get

In [4]:
# JB Buses from GTFS-R URL from Malaysia's Open API
URL = 'https://api.data.gov.my/gtfs-realtime/vehicle-position/mybas-johor'
 
# Parse the GTFS Realtime feed
feed = gtfs_realtime_pb2.FeedMessage()
response = get(URL)
feed.ParseFromString(response.content)
 
# Extract and print vehicle position information
vehicle_positions = [MessageToDict(entity.vehicle) for entity in feed.entity]
print(f'Total vehicles: {len(vehicle_positions)}')
df = pd.json_normalize(vehicle_positions)
print(df)

Total vehicles: 69
     timestamp                        trip.tripId trip.startDate trip.routeId  \
0   1762406447  0002_J31CWL_12514_1_P1_ALLDAY_1_1       20251106       J31CWL   
1   1762406447  0002_J33CWL_12518_1_P1_ALLDAY_1_1       20251106       J33CWL   
2   1762406447  0002_J10CWL_12543_1_A7_ALLDAY_2_2       20251106       J10CWL   
3   1762406447  0002_J13CWL_12547_1_A4_ALLDAY_2_4       20251106       J13CWL   
4   1762406447  0002_J20CWL_12510_1_A7_ALLDAY_2_2       20251106       J20CWL   
..         ...                                ...            ...          ...   
64  1762406447  0002_J40CWL_12522_1_P1_ALLDAY_1_1       20251106       J40CWL   
65  1762406447  0002_J33CWL_12518_1_A6_ALLDAY_2_2       20251106       J33CWL   
66  1762406447  0002_J15CWL_12549_1_P1_ALLDAY_1_1       20251106       J15CWL   
67  1762406447  0002_J21CWL_12529_0_A6_ALLDAY_1_4       20251106       J21CWL   
68  1762406447  0002_J10CWL_12544_0_P3_ALLDAY_1_1       20251106       J10CWL   

    trip

## bus positions - updated every 30s

In [ ]:
# !pip install plotly.express

In [6]:
# Plot longitude and latitude from df on a map
import plotly.express as px

# Ensure columns exist and are named correctly
if 'position.longitude' in df.columns and 'position.latitude' in df.columns:
    fig = px.scatter_mapbox(
        df,
        lat='position.latitude',
        lon='position.longitude',
        hover_name='vehicle.id' if 'vehicle.id' in df.columns else None,
        zoom=10,
        height=600
    )
    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
else:
    print('Longitude and latitude columns not found in df')

C:\Users\darkl\AppData\Local\Temp\ipykernel_29688\96430651.py:6: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


## trains positions - updates every 30s

In [9]:
URL_ktm = 'https://api.data.gov.my/gtfs-realtime/vehicle-position/ktmb'

feed_ktm = gtfs_realtime_pb2.FeedMessage()
response_ktm = get(URL_ktm)
feed_ktm.ParseFromString(response_ktm.content)

ktm_positions = [MessageToDict(entity.vehicle) for entity in feed_ktm.entity]
print(f'Total vehicles: {len(ktm_positions)}')
df_ktm = pd.json_normalize(ktm_positions)
print(df_ktm)


Total vehicles: 8
    timestamp   trip.tripId  position.latitude  position.longitude  \
0  1762406898  weekday_2130           3.257507          101.554825   
1  1762406884  weekday_2027           3.018402          101.719010   
2  1762406870            55           5.344435          102.118050   
3  1762406865          2964           5.824943          100.477936   
4  1762406864          2963           6.023563          100.411514   
5  1762406863          9274           5.107607          100.490950   
6  1762406875  weekday_2134           3.601868          101.591606   
7  1762406879          9027           3.686032          101.516860   

   position.bearing  position.speed    vehicle.id vehicle.label  
0               0.0             0.0  weekday_2130         SCS25  
1               0.0            37.0  weekday_2027         SCS02  
2               0.0            52.0            55         DMU05  
3               0.0            40.0          2964         SCS18  
4               0.0  

In [12]:
# Plot longitude and latitude from df on a map
import plotly.express as px

# Ensure columns exist and are named correctly
if 'position.longitude' in df_ktm.columns and 'position.latitude' in df_ktm.columns:
    fig = px.scatter_mapbox(
    df_ktm,
    lat='position.latitude',
    lon='position.longitude',
    hover_name='vehicle.id' if 'vehicle.id' in df_ktm.columns else None,
    height=600
    )
    fig.update_layout(
    mapbox_style='open-street-map',
    mapbox_center={"lat": 4.5, "lon": 102.0},
    mapbox_zoom=5,
    margin={"r":0,"t":0,"l":0,"b":0}
    )
    fig.show()

else:
    print('Longitude and latitude columns not found in df_ktm')

C:\Users\darkl\AppData\Local\Temp\ipykernel_29688\843018879.py:6: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [13]:
# Real-time vehicle position plotting (updates every 30 seconds)
import time
from IPython.display import clear_output

def plot_realtime_vehicle_positions(url, iterations=10, interval=30):
    for i in range(iterations):
        feed = gtfs_realtime_pb2.FeedMessage()
        response = get(url)
        feed.ParseFromString(response.content)
        vehicle_positions = [MessageToDict(entity.vehicle) for entity in feed.entity]
        df = pd.json_normalize(vehicle_positions)
        clear_output(wait=True)
        print(f'Update {i+1}: Total vehicles: {len(vehicle_positions)}')
        if 'position.longitude' in df.columns and 'position.latitude' in df.columns:
            fig = px.scatter_mapbox(
                df,
                lat='position.latitude',
                lon='position.longitude',
                hover_name='vehicle.id' if 'vehicle.id' in df.columns else None,
                zoom=10,
                height=600
            )
            fig.update_layout(mapbox_style='open-street-map')
            fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            fig.show()
        else:
            print('Longitude and latitude columns not found in df')
        time.sleep(interval)

# Example usage: plot_realtime_vehicle_positions(URL, iterations=5, interval=30)

In [14]:
plot_realtime_vehicle_positions(URL, iterations=5, interval=30)

Update 5: Total vehicles: 70


C:\Users\darkl\AppData\Local\Temp\ipykernel_29688\630250860.py:15: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

